<a href="https://colab.research.google.com/github/OlaOye23/ShopMob/blob/main/App_Functions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install exponent_server_sdk
!pip install --upgrade firebase-admin

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 115 kB 6.9 MB/s 
     |████████████████████████████████| 106 kB 52.4 MB/s 
     |████████████████████████████████| 244 kB 56.1 MB/s 
     |████████████████████████████████| 244 kB 44.7 MB/s 
     |████████████████████████████████| 1.0 MB 50.1 MB/s 
     |████████████████████████████████| 46 kB 3.7 MB/s 
     |████████████████████████████████| 76 kB 4.1 MB/s 
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.17.3
    Uninstalling protobuf-3.17.3:
      Successfully uninstalled protobuf-3.17.3
  Attempting uninstall: google-resumable-media
    Found existing installation: google-resumable-media 0.4.1
    Uninstalling google-resumable-media-0.4.1:
      Successfully uninstalled google-resumable-media-0.4.1
  Attempting uninstall

In [ ]:
#CODEGEN
  
def generate_code(context):
  '''generates and posts a unique 6 digit code (OTP) to a database  
  '''
  code_str= generate_random_no(n_digits= 6)
  print(code_str)
  print("posted to db", post_to_db(code_str))
  return str(code_str)
  
def generate_random_no(n_digits= 6):
  '''uses random numbers to generate a unique n-digit code'''
  import numpy as np

  numbers = []
  for i in range(n_digits):
    numbers.append(int(np.floor(np.random.rand()*10)))
  code = 0
  code_str = ''
  for i in range(0,n_digits,1):
    code += numbers[i]*10**(n_digits-i-1)
    code_str = code_str + str(numbers[i])
  print(code_str)
  return code_str

def post_to_db( code_str):
  '''post code along with user data in code db'''
  #initialise firebase
  import firebase_admin
  from firebase_admin import credentials
  from firebase_admin import firestore

  # Use a service account
  cred = credentials.Certificate('/shopmob/shopmob_service_acc')
  try:
    firebase_admin.initialize_app(cred)
  except:
    print('error occurred in initialising firebase. Perhaps already initialised')

  db = firestore.client()
  #post to db --- uncomment fields to include user details
  data = {
  #u'uname': data['userId'],
  #u'ID': data['userName'],
  u'code': code_str 
        }
  response = db.collection(u'codegen_codes').document(code_str).set(data)
  return response





In [ ]:
generate_code("userId")

242832
242832


FileNotFoundError: ignored

In [ ]:
np.random.rand?

Object `np.random.rand` not found.


In [ ]:
#LIGHTAPP
def get_tokens():
  '''submits push notifications to users stored in a firestore database
  '''
  #initialise firebase
  import firebase_admin
  from firebase_admin import credentials
  from firebase_admin import firestore

  # Use a service account
  cred = credentials.Certificate('/content/shopmob-secret.json')
  try:
    firebase_admin.initialize_app(cred)
  except:
    print('error occurred in initialising firebase. Perhaps already initialised')

  db = firestore.client()
  #get subscribers list
  users_ref = db.collection(u'light_users')
  docs = users_ref.stream()

  uid_ = []
  secret_ = []
  tokens_ = []

  for doc in docs:
    #print(f'{doc.id} => {doc.to_dict()}')
    id = doc.id
    doc = doc.to_dict()
    #print(doc['email'])
    if doc['status'] == "active":
      tokens_.append(doc['token'])

  #send push notifications
  for token in tokens_:
    send_push_message(token,'There is light!', extra= {"":""})

  return doc['token']


  


# Basic arguments. You should extend this function with the push features you
# want to use, or simply pass in a `PushMessage` object.
def send_push_message(token, message, extra=None):
  from exponent_server_sdk import (
    DeviceNotRegisteredError,
    PushClient,
    PushMessage,
    PushServerError,
    PushTicketError,
  )
  from requests.exceptions import ConnectionError, HTTPError
  try:
      response = PushClient().publish(
          PushMessage(to=token,
                      body=message,
                      data=extra))
      print(response)
  except PushServerError as exc:
      # Encountered some likely formatting/validation error.
      rollbar.report_exc_info(
          extra_data={
              'token': token,
              'message': message,
              'extra': extra,
              'errors': exc.errors,
              'response_data': exc.response_data,
          })
      raise
  except (ConnectionError, HTTPError) as exc:
      # Encountered some Connection or HTTP error - retry a few times in
      # case it is transient.
      rollbar.report_exc_info(
          extra_data={'token': token, 'message': message, 'extra': extra})
      raise self.retry(exc=exc)

  try:
      # We got a response back, but we don't know whether it's an error yet.
      # This call raises errors so we can handle them with normal exception
      # flows.
      response.validate_response()
  except DeviceNotRegisteredError:
      # Mark the push token as inactive
      from notifications.models import PushToken
      PushToken.objects.filter(token=token).update(active=False)
  except PushTicketError as exc:
      # Encountered some other per-notification error.
      rollbar.report_exc_info(
          extra_data={
              'token': token,
              'message': message,
              'extra': extra,
              'push_response': exc.push_response._asdict(),
          })
      raise self.retry(exc=exc)




In [ ]:
get_tokens()

PushTicket(push_message=PushMessage(to='ExponentPushToken[SAMX0TGUoZWfA4W1HmPAxJ]', data={'stuff': 'hey'}, title=None, body='Hello', sound=None, ttl=None, expiration=None, priority=None, badge=None, category=None, display_in_foreground=None, channel_id=None, subtitle=None, mutable_content=None), status='ok', message='', details=None, id='740d7b20-6f25-416d-8806-4a12ac2cccd0')
PushTicket(push_message=PushMessage(to='ExponentPushToken[SAMX0TGUoZWfA4W1HmPAxJ]', data={'stuff': 'hey'}, title=None, body='Hello', sound=None, ttl=None, expiration=None, priority=None, badge=None, category=None, display_in_foreground=None, channel_id=None, subtitle=None, mutable_content=None), status='ok', message='', details=None, id='b4e16888-65a2-4029-a22e-bd49b021920a')


'ExponentPushToken[SAMX0TGUoZWfA4W1HmPAxJ]'

In [ ]:
send_push_message('ExponentPushToken[SAMX0TGUoZWfA4W1HmPAxJ]','There is light!')


PushTicket(push_message=PushMessage(to='ExponentPushToken[SAMX0TGUoZWfA4W1HmPAxJ]', data=None, title=None, body='There is light!', sound=None, ttl=None, expiration=None, priority=None, badge=None, category=None, display_in_foreground=None, channel_id=None, subtitle=None, mutable_content=None), status='ok', message='', details=None, id='4811e173-b871-41fc-841d-55896667f053')


FileNotFoundError: ignored

In [ ]:
def light_on_first():#arg= deviceID
  '''post code along with user data in code db'''
  #initialise firebase
  import firebase_admin
  from firebase_admin import credentials
  from firebase_admin import firestore
  import datetime 
  import time

  # Use a service account
  cred = credentials.Certificate('/content/shopmob-secret.json')
  try:
    firebase_admin.initialize_app(cred)
  except:
    print('error occurred in initialising firebase. Perhaps already initialised')

  db = firestore.client()
  now = datetime.datetime.now()
  nowCalc = time.mktime(now.timetuple())
  #post to db
  data = {#deviceID
          u'deviceID': "001",
          u'timeCalc': nowCalc,
          u'location':" [0, 0]",
        }
  response = db.collection(u'light_reports_first').document().set(data)
  return response


def light_on_all():#arg= deviceID
  '''post code along with user data in code db'''
  #initialise firebase
  import firebase_admin
  from firebase_admin import credentials
  from firebase_admin import firestore
  import datetime 
  import time

  # Use a service account
  cred = credentials.Certificate('/content/shopmob-secret.json')
  try:
    firebase_admin.initialize_app(cred)
  except:
    print('error occurred in initialising firebase. Perhaps already initialised')

  db = firestore.client()
  now = datetime.datetime.now()
  nowCalc = time.mktime(now.timetuple())
  #post to db
  data = {#deviceID
          u'deviceID': "001",
          u'timeCalc': nowCalc,
          u'location': "[0, 0]",
        }
  response = db.collection(u'light_reports_all').document().set(data)
  return response



In [ ]:
print(light_on_first())
print(light_on_all())

FileNotFoundError: ignored

In [ ]:
import datetime
datetime.datetime.now()

datetime.datetime(2022, 7, 20, 17, 38, 18, 38229)